
# Tutorial: Introduction to K-Fold Cross-Validation

Di subchapter sebelumnya, kita melihat bahwa satu kali `train_test_split` bisa memberikan hasil yang tidak stabil. Skor evaluasi kita bisa berubah-ubah tergantung pada "keberuntungan" pembagian data secara acak.

**K-Fold Cross-Validation (CV)** adalah solusi standar industri untuk masalah ini. Ini adalah teknik evaluasi yang andal (*robust*) yang memberikan estimasi performa model yang jauh lebih stabil dengan cara melatih dan menguji model pada beberapa subset data yang berbeda.

Di notebook ini, kita akan memahami konsep K-Fold CV dan mengimplementasikannya menggunakan Scikit-learn.




---
### 1. Tujuan Pembelajaran

Di akhir notebook ini, Anda akan dapat:

* Menjelaskan konsep dan alur kerja di balik K-Fold Cross-Validation.
* Memahami mengapa K-Fold CV lebih unggul daripada satu kali `train_test_split`.
* Mengimplementasikan K-Fold CV menggunakan fungsi `cross_val_score` dari Scikit-learn.
* Menginterpretasikan hasil dari *cross-validation*, termasuk rata-rata dan standar deviasi skor.




---
### 2. Konsep Inti: Bagaimana K-Fold Bekerja?

Bayangkan Anda memiliki data dan membaginya menjadi **K "lipatan" (folds)**, misalnya K=5.

Alur kerja K-Fold Cross-Validation adalah sebagai berikut:

1.  **Iterasi 1:** Latih model pada **Fold 2, 3, 4, 5** dan uji pada **Fold 1**. Simpan skornya.
2.  **Iterasi 2:** Latih model pada **Fold 1, 3, 4, 5** dan uji pada **Fold 2**. Simpan skornya.
3.  **Iterasi 3:** Latih model pada **Fold 1, 2, 4, 5** dan uji pada **Fold 3**. Simpan skornya.
4.  **Iterasi 4:** Latih model pada **Fold 1, 2, 3, 5** dan uji pada **Fold 4**. Simpan skornya.
5.  **Iterasi 5:** Latih model pada **Fold 1, 2, 3, 4** dan uji pada **Fold 5**. Simpan skornya.



Setelah 5 iterasi, kita akan memiliki **5 skor performa yang berbeda**. Hasil akhir dari *cross-validation* bukanlah satu skor, melainkan **rata-rata** dan **standar deviasi** dari kelima skor tersebut.

**Keuntungannya:**
* **Lebih Andal:** Rata-rata dari 5 skor memberikan estimasi yang jauh lebih stabil daripada satu skor tunggal.
* **Penggunaan Data yang Efisien:** Setiap titik data digunakan sebagai data pelatihan dan data pengujian tepat satu kali.




---
### 3. Setup: Dataset dan Model yang Sama

Kita akan menggunakan dataset Diabetes dan model Regresi Linear untuk perbandingan yang adil.



In [8]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression

# muat dataset
diabetes = load_diabetes()
X = pd.DataFrame(diabetes.data, columns = diabetes.feature_names)
y = pd.Series(diabetes.target, name = 'DiabetesProgression')

print(X.head())

# model
model = LinearRegression()

        age       sex       bmi        bp        s1        s2        s3  \
0  0.038076  0.050680  0.061696  0.021872 -0.044223 -0.034821 -0.043401   
1 -0.001882 -0.044642 -0.051474 -0.026328 -0.008449 -0.019163  0.074412   
2  0.085299  0.050680  0.044451 -0.005670 -0.045599 -0.034194 -0.032356   
3 -0.089063 -0.044642 -0.011595 -0.036656  0.012191  0.024991 -0.036038   
4  0.005383 -0.044642 -0.036385  0.021872  0.003935  0.015596  0.008142   

         s4        s5        s6  
0 -0.002592  0.019907 -0.017646  
1 -0.039493 -0.068332 -0.092204  
2 -0.002592  0.002861 -0.025930  
3  0.034309  0.022688 -0.009362  
4 -0.002592 -0.031988 -0.046641  



---
### 4. Implementasi dengan `cross_val_score`

Scikit-learn membuat proses ini sangat mudah dengan fungsi `cross_val_score`. Fungsi ini secara otomatis melakukan semua langkah yang dijelaskan di atas.



In [2]:
from sklearn.model_selection import cross_val_score

# jalankan 5 fold cross validation
# cv = 5, K=5
# scoring= r2,
# melatih linearregegression 5 kali
scores = cross_val_score(model,X,y, cv=5, scoring='r2')


---
### 5. Analisis Hasil

Mari kita lihat apa yang dikembalikan oleh `cross_val_score`.



In [3]:
print("Skor R2 dari setiap fold:")
print(scores)

Skor R2 dari setiap fold:
[0.42955615 0.52259939 0.48268054 0.42649776 0.55024834]



Outputnya adalah sebuah array NumPy yang berisi 5 skor R², satu untuk setiap iterasi. Seperti yang bisa kita lihat, skornya bervariasi, sama seperti saat kita mengubah `random_state` sebelumnya.

Namun, sekarang kita bisa mendapatkan gambaran yang lebih lengkap dengan menghitung statistik ringkasannya.



In [4]:
mean_r2 = np.mean(scores)
std_r2 = np.std(scores)

print(f"\nRata-rata R² (dari 5-fold CV): {mean_r2:.4f}")
print(f"Standar Deviasi R²: {std_r2:.4f}")


Rata-rata R² (dari 5-fold CV): 0.4823
Standar Deviasi R²: 0.0493



**Interpretasi:**

* **Rata-rata R² (0.5530):** Ini adalah estimasi performa model kita yang paling mungkin dan paling andal. Daripada melaporkan 0.57 atau 0.59 seperti sebelumnya, kita sekarang memiliki satu angka yang lebih representatif.
* **Standar Deviasi R² (0.0617):** Angka ini memberi tahu kita tentang **stabilitas** model. Standar deviasi yang relatif rendah (dibandingkan dengan rata-ratanya) menunjukkan bahwa performa model tidak terlalu banyak berfluktuasi di berbagai subset data. Standar deviasi yang tinggi akan menjadi tanda bahaya bahwa performa model sangat tidak stabil.

Kita sekarang dapat mengatakan dengan lebih percaya diri:

> "Model Regresi Linear kami diperkirakan memiliki skor R-squared sekitar **0.53 ± 0.06**."

Ini adalah pernyataan yang jauh lebih informatif dan jujur secara statistik daripada hanya melaporkan satu skor tunggal.




---
### 6. Kesimpulan

* **Cross-Validation** adalah standar emas untuk evaluasi model karena memberikan estimasi performa yang **robust (andal) dan stabil**.
* Ia mengatasi masalah "keberuntungan" dari satu kali `train_test_split` dengan mengevaluasi model pada beberapa pembagian data yang berbeda.
* Fungsi `cross_val_score` di Scikit-learn adalah cara yang sangat efisien untuk menerapkan K-Fold CV.
* Selalu laporkan **rata-rata** dan **standar deviasi** dari skor *cross-validation* untuk memberikan gambaran performa yang lengkap.
